In [19]:
# def fit_metrics_print(model, X_train, y_train, X_val, y_val):
#     from sklearn.metrics import accuracy_score, classification_report
#     model.fit(X_train, y_train)
    
#     pred_train = model.predict(X_train)
#     pred_val = model.predict(X_val)
#     print(model)
#     print()
#     print('train accuracy :', accuracy_score(y_train, pred_train))
#     print('validation accuracy :', accuracy_score(y_val, pred_val))
#     print('='*60)

In [234]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [235]:
X = pd.read_csv('../data/ohe_edit_4.8.1(add_col).csv')
X.drop(columns='target', inplace=True)
y = pd.read_csv('../data/ohe_edit_4.8.1(add_col).csv')['target']
X_test = pd.read_csv('../data/prepcd_test.csv')

#### 우상님 코드용

In [236]:
# X.rename(columns={'company_size_<10':'company_size_~10'},inplace=True)

# X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, shuffle=True, random_state=1)
# X_train.shape, X_val.shape, y_train.shape, y_val.shape

#### 내코드용 1차

In [237]:
# X = pd.read_csv('../data/prepcd_train.csv')
# X.drop(columns='target', inplace=True)
# y = pd.read_csv('../data/prepcd_train.csv')['target']
# X_test = pd.read_csv('../data/prepcd_test.csv')

#### 내코드용 2차

In [238]:
X = pd.read_csv('../data/prepcd_lbe_train.csv')
X.drop(columns='target', inplace=True)
y = pd.read_csv('../data/prepcd_lbe_train.csv')['target']
X_test = pd.read_csv('../data/prepcd_lbe_test.csv')

### 우상님 최종본

In [239]:
# X = pd.read_csv('../data/final.csv')
# X.drop(columns='target', inplace=True)
# y = pd.read_csv('../data/final.csv')['target']
# X_test = pd.read_csv('../data/final.csv')

In [240]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, shuffle=True, random_state=1)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((14368, 13), (4790, 13), (14368,), (4790,))

### Polynomial 컬럼 추가(Trial&Error) XGBoost모델 feature_importances_ 기준

### 중요도 컬럼뽑기 - 랜덤서치(nopoly,no oversampling) 한번 실행후

In [162]:
imprt_lst = pd.Series(be.feature_importances_, index=X_train.columns).sort_values(ascending=False)
to_poly_lst = list(imprt_lst[imprt_lst>imprt_lst.quantile(q=0.8)].index)
to_poly_lst

['city_development_index', 'relevent_experience', 'enrolled_university']

In [163]:
from sklearn.preprocessing import PolynomialFeatures

poly_f = PolynomialFeatures(degree=2, include_bias=False)
poly_f1 = PolynomialFeatures(degree=2, include_bias=False)

poly_train = poly_f.fit_transform(X.loc[:,X.columns.isin(to_poly_lst)])
poly_test = poly_f1.fit_transform(X_test.loc[:,X_test.columns.isin(to_poly_lst)])

X_train_poly_tomerge = pd.DataFrame(poly_train, columns=poly_f.get_feature_names())
X_test_poly_tomerge = pd.DataFrame(poly_test, columns=poly_f1.get_feature_names())

X_train_poly = pd.concat([X, X_train_poly_tomerge], axis=1)
X_test_poly = pd.concat([X_test, X_test_poly_tomerge], axis=1)

In [164]:
vars_poly = {f'x{i}':v for i,v in enumerate(to_poly_lst)}
vars_poly

{'x0': 'city_development_index',
 'x1': 'relevent_experience',
 'x2': 'enrolled_university'}

In [165]:
X_train_poly, X_val_poly, y_train_poly, y_val_poly = train_test_split(X_train_poly, y, stratify=y, shuffle=True, random_state=1)
X_train_poly.shape, X_val_poly.shape, y_train_poly.shape, y_val_poly.shape

((14368, 22), (4790, 22), (14368,), (4790,))

In [166]:
X_train_poly

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,...,training_hours,x0,x1,x2,x0^2,x0 x1,x0 x2,x1^2,x1 x2,x2^2
12751,18294,83,0.762,1,1,0.0,2.0,0,2.0,3.0,...,192,0.762,1.0,0.0,0.580644,0.762,0.00,1.0,0.0,0.0
12061,14050,64,0.624,1,0,0.0,2.0,0,12.0,4.0,...,3,0.624,0.0,0.0,0.389376,0.000,0.00,0.0,0.0,0.0
18919,720,49,0.920,1,1,2.0,2.0,0,13.0,3.0,...,52,0.920,1.0,2.0,0.846400,0.920,1.84,1.0,2.0,4.0
3215,24912,64,0.624,1,1,0.0,2.0,0,5.0,7.0,...,17,0.624,1.0,0.0,0.389376,0.624,0.00,1.0,0.0,0.0
5273,3992,49,0.920,1,0,0.0,2.0,0,4.0,2.0,...,204,0.920,0.0,0.0,0.846400,0.000,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,15912,5,0.920,0,1,0.0,2.0,1,4.0,0.0,...,166,0.920,1.0,0.0,0.846400,0.920,0.00,1.0,0.0,0.0
8701,1740,5,0.920,1,0,0.0,2.0,0,18.0,2.0,...,26,0.920,0.0,0.0,0.846400,0.000,0.00,0.0,0.0,0.0
18883,17805,65,0.899,1,0,0.0,2.0,0,21.0,4.0,...,24,0.899,0.0,0.0,0.808201,0.000,0.00,0.0,0.0,0.0
17051,9115,50,0.767,1,0,0.0,2.0,0,8.0,0.0,...,165,0.767,0.0,0.0,0.588289,0.000,0.00,0.0,0.0,0.0


### 오버샘플링

In [167]:
# !pip install imblearn

In [168]:
# from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

In [169]:
# oversample = SMOTE()
# sm = SMOTE(ratio='auto', kind='regular')
sm = SMOTE(random_state=0)
X_train_ovsp, y_train_ovsp = sm.fit_resample(X_train, list(y_train))

In [170]:
sm = SMOTE(random_state=0)
X_train_poly_ovsp, y_train_poly_ovsp = sm.fit_resample(X_train_poly, list(y_train_poly))

In [171]:
X_train_ovsp.shape, X_train.shape, X_train_poly_ovsp.shape, X_train_poly.shape

((21570, 13), (14368, 13), (21570, 22), (14368, 22))

- - -
# XGBoost_randomized

In [241]:
param = dict(
    n_estimators=[100, 200, 300, 400, 500],
    learning_rate=[0.001,0.005,0.01,0.05,0.1,0.5],
    max_depth=range(1, 7)
)

In [242]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=200,
                   learning_rate=0.5,
                   max_depth=2,
                   random_state=1)
xgb.fit(X_train, y_train)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:07:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=12, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

#### nopoly-nooversampling

In [243]:
random_xgb = RandomizedSearchCV(xgb,
                             param_distributions=param,
                             n_iter=10,
                             scoring='accuracy',
                             cv=3,
                             n_jobs=-1,
                             random_state=1
                             )
random_xgb.fit(X_train, y_train)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:08:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.5, max_delta_step=0,
                                           max_depth=2, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=200, n_jobs=12,
                                           num_parallel_tree=1, random_state=1,
                                           reg_alpha=0, reg_lambda=1,
                                           sca

In [244]:
be = random_xgb.best_estimator_
pd.Series(be.feature_importances_, index=X_train.columns).sort_values(ascending=False)

city_development_index    0.306455
company_size              0.283693
education_level           0.095922
company_type              0.053634
relevent_experience       0.043409
last_new_job              0.034252
enrolled_university       0.032108
experience                0.031688
city                      0.029302
major_discipline          0.024663
enrollee_id               0.022918
gender                    0.021100
training_hours            0.020859
dtype: float32

In [245]:
pd.DataFrame(random_xgb.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
8,3.040497,0.025913,0.009295,0.002757,100,5,0.05,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",0.797704,0.789935,0.794112,0.793917,0.003174,1
5,13.078803,0.017108,0.013333,0.000924,500,5,0.005,"{'n_estimators': 500, 'max_depth': 5, 'learnin...",0.798330,0.789309,0.790353,0.792664,0.004029,2
0,8.197504,0.169728,0.018726,0.004010,200,6,0.05,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.794572,0.790144,0.789726,0.791481,0.002192,3
1,9.854167,0.041715,0.015151,0.002390,300,5,0.05,"{'n_estimators': 300, 'max_depth': 5, 'learnin...",0.788935,0.789726,0.790979,0.789880,0.000842,4
2,5.749084,0.044939,0.013183,0.002429,500,2,0.05,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.792276,0.787430,0.784715,0.788140,0.003127,5
9,2.968959,0.049378,0.007820,0.000399,300,2,0.1,"{'n_estimators': 300, 'max_depth': 2, 'learnin...",0.790605,0.786803,0.784089,0.787166,0.002673,6
4,1.156509,0.018084,0.008134,0.001659,100,2,0.005,"{'n_estimators': 100, 'max_depth': 2, 'learnin...",0.782672,0.780956,0.784089,0.782572,0.001281,7
7,9.161443,0.147774,0.018074,0.002108,500,4,0.001,"{'n_estimators': 500, 'max_depth': 4, 'learnin...",0.782672,0.779703,0.784715,0.782364,0.002058,8
3,3.382820,0.010760,0.012545,0.001839,200,3,0.5,"{'n_estimators': 200, 'max_depth': 3, 'learnin...",0.780376,0.779495,0.785968,0.781946,0.002866,9
6,5.871087,0.018682,0.013183,0.001994,500,2,0.01,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.780793,0.782418,0.781165,0.781459,0.000695,10


In [246]:
pred_train = random_xgb.predict(X_train)
pred_val = random_xgb.predict(X_val)

print('train accuracy :', accuracy_score(y_train, pred_train))
print('validation accuracy :', accuracy_score(y_val, pred_val))

train accuracy : 0.8159103563474388
validation accuracy : 0.7954070981210856


### 성능지표 출력 및 

In [251]:
score_df = []
score_df = get_scores(score_df, random_xgb, X_val, y_val, "random_xgb")
score_df

In [222]:
y_pred_proba = random_xgb.predict_proba(X_val)[:,1]

In [223]:
from sklearn.metrics import roc_auc_score, f1_score, plot_confusion_matrix, \
    plot_roc_curve, plot_precision_recall_curve, classification_report, precision_score, recall_score

In [225]:
df_report = pd.DataFrame(columns={'ROC-AUC'}, data=[0])
df_report['ROC-AUC'] = roc_auc_score(y_val, y_pred_proba)
df_report['F1'] = f1_score(y_val, pred_val)
df_report['precision_0'] = precision_score(y_val, pred_val, pos_label=0)
df_report['recall_0'] = recall_score(y_val, pred_val, pos_label=0)
df_report['precision_1'] = precision_score(y_val, pred_val, pos_label=1)
df_report['recall_1'] = recall_score(y_val, pred_val, pos_label=1)

df_report.index = ['XGBClassifier']
df_report

,ROC-AUC,F1,precision_0,recall_0,precision_1,recall_1
XGBClassifier,0.807359,0.583333,0.860132,0.868743,0.592401,0.574539


In [227]:
def get_scores(report_df, model, X_test, y_test, name):    
    report = pd.DataFrame(columns={'ROC-AUC'}, data=[0])
    report['ROC-AUC'] = roc_auc_score(y_test,model.predict_proba(X_test)[:, 1])
    report['F1'] = f1_score(y_test, model.predict(X_test))
    report['precision_0'] = precision_score(
        y_test, model.predict(X_test), pos_label=0)
    report['precision_1'] = precision_score(
        y_test, model.predict(X_test), pos_label=1)
    report['recall_0'] = recall_score(
        y_test, model.predict(X_test), pos_label=0)
    report['recall_1'] = recall_score(
        y_test, model.predict(X_test), pos_label=1)

    report.index = [name]
    report_df = report_df.append(report)
    return report_df

In [ ]:
# ROC curve 그리는 코드

In [1]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

plot_precision_recall_curve(xgb2, X_test, y_test, ax=ax1)
plot_roc_curve(xgb2, X_test, y_test, ax=ax2)

NameError: name 'plt' is not defined

#### poly-nooversampling

In [204]:
random_xgb_poly = RandomizedSearchCV(xgb,
                             param_distributions=param,
                             n_iter=10,
                             scoring='accuracy',
                             cv=3,
                             n_jobs=-1,
                             random_state=1
                             )
random_xgb_poly.fit(X_train_poly, y_train_poly)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:01:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.5, max_delta_step=0,
                                           max_depth=2, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=200, n_jobs=12,
                                           num_parallel_tree=1, random_state=1,
                                           reg_alpha=0, reg_lambda=1,
                                           sca

In [205]:
be_poly = random_xgb_poly.best_estimator_
pd.Series(be_poly.feature_importances_, index=X_train_poly.columns).sort_values(ascending=False)

x1 x2                     0.345908
city_development_index    0.260006
relevent_experience       0.183445
x0 x2                     0.036809
education_level           0.028735
enrolled_university       0.023216
city                      0.021890
x0 x1                     0.017378
experience                0.016550
last_new_job              0.014491
company_size              0.012117
enrollee_id               0.010652
training_hours            0.009614
company_type              0.008924
major_discipline          0.007726
gender                    0.002540
x0                        0.000000
x1                        0.000000
x2                        0.000000
x0^2                      0.000000
x1^2                      0.000000
x2^2                      0.000000
dtype: float32

In [206]:
pd.DataFrame(random_xgb_poly.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
2,4.713576,0.022330,0.018083,0.002100,500,2,0.05,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.789979,0.783671,0.777615,0.783755,0.005048,1
9,1.942920,0.012197,0.011390,0.000824,300,2,0.1,"{'n_estimators': 300, 'max_depth': 2, 'learnin...",0.788518,0.782209,0.777198,0.782642,0.004631,2
4,1.028531,0.007600,0.010266,0.000797,100,2,0.005,"{'n_estimators': 100, 'max_depth': 2, 'learnin...",0.782881,0.780956,0.783880,0.782572,0.001213,3
6,4.681373,0.012359,0.013348,0.000618,500,2,0.01,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.783925,0.781583,0.781165,0.782224,0.001215,4
7,6.762642,0.058373,0.012206,0.002079,500,4,0.001,"{'n_estimators': 500, 'max_depth': 4, 'learnin...",0.782672,0.779286,0.781583,0.781180,0.001411,5
5,9.835636,0.044849,0.012548,0.000466,500,5,0.005,"{'n_estimators': 500, 'max_depth': 5, 'learnin...",0.786221,0.779077,0.776989,0.780762,0.003953,6
8,2.623124,0.012451,0.008796,0.001049,100,5,0.05,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",0.785386,0.781165,0.774692,0.780414,0.004398,7
1,7.697340,0.063592,0.013519,0.002005,300,5,0.05,"{'n_estimators': 300, 'max_depth': 5, 'learnin...",0.782046,0.775945,0.770933,0.776308,0.004544,8
0,6.506692,0.069699,0.011230,0.001049,200,6,0.05,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.775783,0.776989,0.772395,0.775056,0.001945,9
3,2.664271,0.013145,0.017255,0.000615,200,3,0.5,"{'n_estimators': 200, 'max_depth': 3, 'learnin...",0.771608,0.767384,0.760702,0.766564,0.004490,10


In [207]:
pred_train_poly = random_xgb_poly.predict(X_train_poly)
pred_val_poly = random_xgb_poly.predict(X_val_poly)

print('train accuracy :', accuracy_score(y_train_poly, pred_train_poly))
print('validation accuracy :', accuracy_score(y_val_poly, pred_val_poly))

train accuracy : 0.7887667037861915
validation accuracy : 0.7791231732776618


#### nopoly-oversampling

In [208]:
random_xgb_ovsp = RandomizedSearchCV(xgb,
                             param_distributions=param,
                             n_iter=10,
                             scoring='accuracy',
                             cv=3,
                             n_jobs=-1,
                             random_state=1
                             )
random_xgb_ovsp.fit(X_train_ovsp, y_train_ovsp)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:02:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.5, max_delta_step=0,
                                           max_depth=2, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=200, n_jobs=12,
                                           num_parallel_tree=1, random_state=1,
                                           reg_alpha=0, reg_lambda=1,
                                           sca

In [209]:
be_ovsp = random_xgb_ovsp.best_estimator_
pd.Series(be_ovsp.feature_importances_, index=X_train_ovsp.columns).sort_values(ascending=False)

enrolled_university       0.281953
city_development_index    0.147921
education_level           0.140914
last_new_job              0.117147
company_type              0.103675
company_size              0.065948
city                      0.041969
gender                    0.027233
experience                0.022011
relevent_experience       0.020783
enrollee_id               0.011491
training_hours            0.010002
major_discipline          0.008953
dtype: float32

In [210]:
pd.DataFrame(random_xgb_ovsp.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
1,9.824725,0.272670,0.014640,0.001380,300,5,0.05,"{'n_estimators': 300, 'max_depth': 5, 'learnin...",0.501669,0.918637,0.930042,0.783449,0.199303,1
0,8.046455,0.677981,0.014640,0.000797,200,6,0.05,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.501808,0.919889,0.927677,0.783125,0.198946,2
3,3.577685,0.044657,0.013664,0.002391,200,3,0.5,"{'n_estimators': 200, 'max_depth': 3, 'learnin...",0.502364,0.916134,0.924061,0.780853,0.196948,3
9,2.755569,0.025204,0.010411,0.000460,300,2,0.1,"{'n_estimators': 300, 'max_depth': 2, 'learnin...",0.501669,0.912517,0.926704,0.780297,0.197105,4
2,6.249968,0.079200,0.013989,0.001218,500,2,0.05,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.501947,0.907789,0.923505,0.777747,0.195125,5
8,3.147269,0.028780,0.010411,0.000920,100,5,0.05,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",0.507789,0.897079,0.914465,0.773111,0.187745,6
5,11.937436,0.035603,0.021147,0.003680,500,5,0.005,"{'n_estimators': 500, 'max_depth': 5, 'learnin...",0.549513,0.866481,0.889013,0.768336,0.155004,7
6,5.566443,0.018473,0.014640,0.002391,500,2,0.01,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.594159,0.827121,0.845480,0.755586,0.114393,8
7,8.668167,0.373522,0.015291,0.000460,500,4,0.001,"{'n_estimators': 500, 'max_depth': 4, 'learnin...",0.625035,0.810570,0.827399,0.754335,0.091687,9
4,1.430163,0.015211,0.011387,0.001217,100,2,0.005,"{'n_estimators': 100, 'max_depth': 2, 'learnin...",0.641307,0.724618,0.733936,0.699954,0.041643,10


In [211]:
pred_train_ovsp = random_xgb_ovsp.predict(X_train_ovsp)
pred_val_ovsp = random_xgb_ovsp.predict(X_val)

print('train accuracy :', accuracy_score(y_train_ovsp, pred_train_ovsp))
print('validation accuracy :', accuracy_score(y_val, pred_val_ovsp))

train accuracy : 0.8717199814557255
validation accuracy : 0.4102296450939457


#### poly-oversampleling

In [212]:
random_xgb_poly_ovsp = RandomizedSearchCV(xgb,
                             param_distributions=param,
                             n_iter=10,
                             scoring='accuracy',
                             cv=3,
                             n_jobs=-1,
                             random_state=1
                             )
random_xgb_poly_ovsp.fit(X_train_poly_ovsp, y_train_poly_ovsp)

c:\users\inje.jeong\anaconda3\envs\ml\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:03:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.5, max_delta_step=0,
                                           max_depth=2, min_child_weight=1,
                                           missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=200, n_jobs=12,
                                           num_parallel_tree=1, random_state=1,
                                           reg_alpha=0, reg_lambda=1,
                                           sca

In [213]:
be_poly_ovsp = random_xgb_poly_ovsp.best_estimator_
pd.Series(be_poly_ovsp.feature_importances_, index=X_train_poly_ovsp.columns).sort_values(ascending=False)

relevent_experience       0.310106
x1                        0.142896
enrolled_university       0.134845
city_development_index    0.102941
education_level           0.071293
last_new_job              0.042070
x1 x2                     0.040505
company_type              0.037511
company_size              0.027264
x2^2                      0.020089
x0^2                      0.017557
city                      0.014445
experience                0.007476
gender                    0.006362
x0 x2                     0.005338
enrollee_id               0.005173
major_discipline          0.004925
x0 x1                     0.004637
training_hours            0.004565
x0                        0.000000
x2                        0.000000
x1^2                      0.000000
dtype: float32

In [214]:
pd.DataFrame(random_xgb_poly_ovsp.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
1,12.811375,0.453859,0.018879,0.002040,300,5,0.05,"{'n_estimators': 300, 'max_depth': 5, 'learnin...",0.501669,0.917803,0.928651,0.782707,0.198774,1
3,4.698781,0.069332,0.015941,0.000460,200,3,0.5,"{'n_estimators': 200, 'max_depth': 3, 'learnin...",0.502225,0.918776,0.924339,0.781780,0.197688,2
9,3.684741,0.038869,0.012045,0.000598,300,2,0.1,"{'n_estimators': 300, 'max_depth': 2, 'learnin...",0.501669,0.917246,0.925035,0.781317,0.197766,3
0,10.419325,0.652659,0.018243,0.002343,200,6,0.05,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.501669,0.916968,0.924618,0.781085,0.197602,4
2,8.158308,0.046336,0.016604,0.002619,500,2,0.05,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.501669,0.914604,0.922392,0.779555,0.196521,5
5,16.559726,0.049356,0.022799,0.003445,500,5,0.005,"{'n_estimators': 500, 'max_depth': 5, 'learnin...",0.539917,0.883449,0.897636,0.773667,0.165388,6
8,4.321545,0.030492,0.013664,0.001594,100,5,0.05,"{'n_estimators': 100, 'max_depth': 5, 'learnin...",0.507371,0.898748,0.910570,0.772230,0.187345,7
6,7.620798,0.029774,0.019375,0.003931,500,2,0.01,"{'n_estimators': 500, 'max_depth': 2, 'learnin...",0.574548,0.853825,0.863978,0.764117,0.134109,8
7,12.233893,0.172490,0.018728,0.001214,500,4,0.001,"{'n_estimators': 500, 'max_depth': 4, 'learnin...",0.623922,0.824757,0.842698,0.763792,0.099174,9
4,1.865458,0.034519,0.011387,0.000460,100,2,0.005,"{'n_estimators': 100, 'max_depth': 2, 'learnin...",0.661752,0.742003,0.744784,0.716180,0.038503,10


In [215]:
pred_train_poly_ovsp = random_xgb_poly_ovsp.predict(X_train_poly_ovsp)
pred_val_poly_ovsp = random_xgb_poly_ovsp.predict(X_val_poly)

print('train accuracy :', accuracy_score(y_train_poly_ovsp, pred_train_poly_ovsp))
print('validation accuracy :', accuracy_score(y_val, pred_val_poly_ovsp))

train accuracy : 0.8720445062586927
validation accuracy : 0.7753653444676409
